In [2]:
import os
import re
import legacy
import click
import dnnlib
import numpy as np
import PIL.Image
import torch

from sklearn.decomposition import PCA
from torch_utils import misc
from torch_utils import persistence
from torch_utils.ops import conv2d_resample
from torch_utils.ops import upfirdn2d
from torch_utils.ops import bias_act
from torch_utils.ops import fma
from typing import List, Optional

In [3]:
def generate_from_seed(
    network_pkl: str,
    seeds: [int], 
    expanded_seeds: [List[int]],
    outdir: str
):
    
    print('Loading networks from "%s"...' % network_pkl)
    device = torch.device('cuda')
    print(network_pkl)
    with dnnlib.util.open_url(network_pkl) as f:
        G = legacy.load_network_pkl(f)['G_ema'].to(device)  # type: ignore

    os.makedirs(outdir, exist_ok=True)
    
    if seeds is None:
        ctx.fail('--seeds option is required when not using --projected-w')

    # Labels.
    class_idx = None
    label = torch.zeros([1, G.c_dim], device=device)
    if G.c_dim != 0:
        if class_idx is None:
            ctx.fail('Must specify class label with --class when using a conditional network')
        label[:, class_idx] = 1
    else:
        if class_idx is not None:
            print('warn: --class=lbl ignored when running on an unconditional network')
            
    # Generate images.
    for seed_idx, seed in enumerate(expanded_seeds):
        print('Generating image for seed %d (%d/%d) ...' % (seeds[seed_idx], seed_idx, len(seeds)))
        z = torch.from_numpy(seed).to(device)
        img = G(z, label, truncation_psi = 0.5, noise_mode = 'const')
        img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
        PIL.Image.fromarray(img[0].cpu().numpy(), 'RGB').save(f'{outdir}/seed{seeds[seed_idx]:04d}.png')

In [4]:
url = 'D:\Rutgers_Study_Files\\2021Fall\\535\\final_project\\network-snapshot-010483.pkl'
device = torch.device('cuda')
outdir_1 = 'D:\\Rutgers_Study_Files\\2021Fall\\535\\final_project\\generated'
outdir_2 = 'D:\\Rutgers_Study_Files\\2021Fall\\535\\final_project\\generated_2'

In [5]:
def expand_seed(seeds):
    result = []
    vector_size = 512
    for seed in seeds:
        rnd = np.random.RandomState(seed)
        result.append( rnd.randn(1, vector_size) )
    return np.array(result)

In [6]:
def generate_seeds(size):
    seeds = []
    for i in range(size):
        z = 200+i
        seeds.append(z)
    expanded_seeds = expand_seed(seeds)
    return expanded_seeds, seeds

In [7]:
expanded_seeds, seeds = generate_seeds(10)
expanded_seeds.shape

(10, 1, 512)

In [8]:
def changeDimDown(expanded_seeds):
    res = []
    for i in range(len(expanded_seeds)):
        res.append(expanded_seeds[i][0])
    return np.array(res)

In [9]:
seeds_down = changeDimDown(expanded_seeds)
seeds_down.shape

(10, 512)

In [10]:
def changeDimUp(arr):
    res = []
    for i in range(len(arr)):
        res.append([arr[i]])
    return np.array(res)

In [11]:
pca = PCA(n_components=10)
pca.fit(seeds_down)
_x_ = pca.explained_variance_ratio_
_ = pca.transform(seeds_down)
pca_unchanged = pca.inverse_transform(_)
_[0:100, 3:5] = 30
pca_changed = pca.inverse_transform(_)
first = changeDimUp(pca_unchanged)
second = changeDimUp(pca_changed)
_x_

array([1.37603420e-01, 1.22300691e-01, 1.18371650e-01, 1.17925797e-01,
       1.10009472e-01, 1.06734735e-01, 1.05160558e-01, 9.55104617e-02,
       8.63832137e-02, 8.91828216e-33])

In [12]:
generate_from_seed(url, seeds, first, outdir_1)

Loading networks from "D:\Rutgers_Study_Files\2021Fall\535\final_project\network-snapshot-010483.pkl"...
D:\Rutgers_Study_Files\2021Fall\535\final_project\network-snapshot-010483.pkl
Generating image for seed 200 (0/10) ...
Setting up PyTorch plugin "bias_act_plugin"... Failed!


C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py:50: UserWarning: Failed to build CUDA kernels for bias_act. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\bias_act.py", line 48, in _init
    _plugin = custom_ops.get_plugin('bias_act_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 64, in get_plugin
    raise RuntimeError(f'Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "{__file__}".')
RuntimeError: Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py".

  warnings.warn('Failed to build CUDA kernels for bias_act. Falling back to slow reference implementation. Details:\n\n' + traceback.form

Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... 

C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 64, in get_plugin
    raise RuntimeError(f'Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "{__file__}".')
RuntimeError: Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py".

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback

Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... 

...

C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 64, in get_plugin
    raise RuntimeError(f'Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "{__file__}".')
RuntimeError: Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py".

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback

Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... 

C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 64, in get_plugin
    raise RuntimeError(f'Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "{__file__}".')
RuntimeError: Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py".

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback

Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
torch.Size([1, 256, 256, 3]) hlkhjkhkj


C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 64, in get_plugin
    raise RuntimeError(f'Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "{__file__}".')
RuntimeError: Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py".

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback

In [107]:
generate_from_seed(url, seeds, second, outdir_2)

Loading networks from "D:\Rutgers_Study_Files\2021Fall\535\final_project\network-snapshot-010483.pkl"...
D:\Rutgers_Study_Files\2021Fall\535\final_project\network-snapshot-010483.pkl
(1, 512)
Generating image for seed 200 (0/100) ...
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!


C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 64, in get_plugin
    raise RuntimeError(f'Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "{__file__}".')
RuntimeError: Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py".

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback

Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... 

C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 64, in get_plugin
    raise RuntimeError(f'Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "{__file__}".')
RuntimeError: Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py".

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback

Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... 

...

C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 64, in get_plugin
    raise RuntimeError(f'Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "{__file__}".')
RuntimeError: Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py".

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback

Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... 

C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 64, in get_plugin
    raise RuntimeError(f'Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "{__file__}".')
RuntimeError: Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py".

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback

Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!
Setting up PyTorch plugin "upfirdn2d_plugin"... Failed!


C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py:34: UserWarning: Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:

Traceback (most recent call last):
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\ops\upfirdn2d.py", line 32, in _init
    _plugin = custom_ops.get_plugin('upfirdn2d_plugin', sources=sources, extra_cuda_cflags=['--use_fast_math'])
  File "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py", line 64, in get_plugin
    raise RuntimeError(f'Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "{__file__}".')
RuntimeError: Could not find MSVC/GCC/CLANG installation on this computer. Check _find_compiler_bindir() in "C:\Users\cszha\styleGAN\stylegan2-ada-pytorch\torch_utils\custom_ops.py".

  warnings.warn('Failed to build CUDA kernels for upfirdn2d. Falling back to slow reference implementation. Details:\n\n' + traceback